In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
import streamlit as st



In [29]:
data = pd.read_excel("cars.xls")

In [30]:
data.shape

(804, 12)

In [31]:
data.head(5)

,Price,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,17314.103129,8221,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,1
1,17542.036083,9135,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
2,16218.847862,13196,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
3,16336.913140,16342,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,0
4,16339.170324,19832,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,1


In [32]:
data["Make"].unique()

array(['Buick', 'Cadillac', 'Chevrolet', 'Pontiac', 'SAAB', 'Saturn'],
      dtype=object)

In [33]:
df = data.copy()

In [37]:
df = df.drop(columns=["Leather","Sound","Cruise"])


In [38]:
df.shape

(804, 9)

In [39]:
X = df.drop("Price", axis=1)
y = df["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Price     804 non-null    float64
 1   Mileage   804 non-null    int64  
 2   Make      804 non-null    object 
 3   Model     804 non-null    object 
 4   Trim      804 non-null    object 
 5   Type      804 non-null    object 
 6   Cylinder  804 non-null    int64  
 7   Liter     804 non-null    float64
 8   Doors     804 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 56.7+ KB


- pop line -- one code encoing gibi dönüşümlerini standart yapmamızı sağlar

In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Preprocessing pipeline
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), ["Doors", "Cylinder", "Mileage", "Liter"]),
        ("cat", OneHotEncoder(), ["Make", "Model", "Trim", "Type"])
    ]
)


In [42]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

my_model = LinearRegression()

pipe = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', my_model)
])

pipe.fit(X_train, y_train)



Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Doors', 'Cylinder',
                                                   'Mileage', 'Liter']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Make', 'Model', 'Trim',
                                                   'Type'])])),
                ('model', LinearRegression())])

In [43]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = pipe.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R2 Score:", r2)


Mean Squared Error: 697393.1791110976
R2 Score: 0.9912072832405728


* Stremlit ile modeli yayma/deploy/ kullanıma sunma


In [44]:
# Tahmin yapmak için girdileri al
input_data = pd.DataFrame({
    'Doors': [doors],
    'Cylinder': [cylinder],
    'Mileage': [mileage],
    'Liter': [liter],
    'Make': [make],
    'Model': [model_input],
    'Trim': [trim],
    'Type': [car_type]
})

# Kategorik verileri sayısal forma dönüştürme
input_data_encoded = pd.get_dummies(input_data)

# Eksik sütunları tamamlayalım
missing_cols = set(X_train.columns) - set(input_data_encoded.columns)
for col in missing_cols:
    input_data_encoded[col] = 0

# Sıralamayı düzenleyelim
input_data_encoded = input_data_encoded[X_train.columns]

# Tahmin yap
prediction = my_model.predict(input_data_encoded)

st.subheader('Prediction:')
st.write('Predicted Price:', prediction[0])


2024-06-11 19:20:18.538 
  command:

    streamlit run C:\Users\Osman\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-06-11 19:20:18.540 Session state does not function when running a script without `streamlit run`
c:\Users\Osman\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'Toyota'